just experimetns/trials

# Imports and preparation

def count_parameters(model):
    """
    Conta o número de parâmetros treináveis em um modelo PyTorch.

    Args:
        model (torch.nn.Module): O modelo PyTorch a ser avaliado.

    Returns:
        int: O número de parâmetros treináveis do modelo.

    """
    # Retorna a soma do número de elementos em cada tensor de parâmetro que requer gradiente.
    # A propriedade "requires_grad" é definida como True para todos os tensores de parâmetro que
    # precisam ser treinados, enquanto que para aqueles que não precisam, ela é definida como False.
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
import torch

In [1]:
from haystack.nodes import PromptNode


OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [ ]:
# Initialize the node:
prompt_node = PromptNode()


In [ ]:
# Run a prompt
prompt_node("What is the capital of Germany?")

# Here's the output:
['berlin']

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [30]:
import time, gc

In [6]:
from util import util_elastic

In [7]:
from util import util_search

In [48]:
from haystack.nodes import SentenceTransformersRankerLimit
from haystack.nodes import MonoT5RankerLimit 

In [10]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [11]:
PATH_QUERY = '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

In [12]:
PATH_SEARCH_EXPERIMENT =  '../data/search/juris_tcu_index/search_experiment.csv'
PATH_SEARCH_RESULT =  '../data/search/juris_tcu_index/search_experiment_result.csv'

# Create reference to index (elastic search)

Documentation https://docs.haystack.deepset.ai/docs
https://haystack.deepset.ai/tutorials/09_dpr_training 


In [13]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu_index
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_index', 'uuid': 'pkdKp2wmTAOM0HyrusfENQ', 'pri': '1', 'rep': '1', 'docs.count': '13252', 'docs.deleted': '0', 'store.size': '276.4mb', 'pri.store.size': '276.4mb'}

There are no index with name indir%


In [16]:
INDEX_NAME = 'indir_juris_tcu_index'

In [19]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13252

Qtd de embeddings 13252

Documento.id=1: <Document: id=1, content='O termo é "Abandono de cargo".
Abandono de cargo tem definição: "Configura abandono de cargo a ausên...'>


# Load search pipelines

In [34]:
# Test query
parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou."

In [32]:
# defining retriever BM25
pipeline_bm25 = util_elastic.return_pipeline_bm25(index)


In [37]:
doc_search_return = pipeline_bm25.run(
    query=parm_query,
    params={
        "Retriever": {
            "top_k": 10,
            "filters":{"class": ["Termo"]}
        }})
util_search.detail_document_found(doc_search_return)


Parâmetros usados: {'Retriever': {'top_k': 10, 'filters': {'class': ['Termo']}}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=1102005, content='O termo é "Plano de contratações".
Plano de contratações tem definição: "É o documento no qual a org...'>

Último (10):
<Document: id=6627, content='O termo é "Matriz de fiscalização".
Matriz de fiscalização tem definição: "Uma matriz de fiscalizaçã...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Plano de contratações', 0.9857202004884205]
1 : ['Competência do TCU', 0.9850136948440441]
2 : ['Teletrabalho', 0.9823720344893851]
3 : ['Nexo de causalidade', 0.9818282753435749]
4 : ['Processo administrativo de responsabilização', 0.9816628569708713]
5 : ['Ofício de requisiç

In [41]:
## defining retriever STS (sentence similarity)
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
# requer Embedding dimensions of the model (1024) don't match the embedding dimensions of the document store (768). Initiate ElasticsearchDocumentStore again with arg embedding_dim=1024.
# nome_modelo = "unicamp-dl/ptt5-base-pt-msmarco-100k-v2"
# nome_modelo = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"

In [43]:
pipeline_sts = util_elastic.return_pipeline_sts(index, nome_caminho_modelo)

In [44]:
doc_search_return = pipeline_sts.run(
    query=parm_query,
    params={
        "Retriever": {
            "top_k": 10,
            "filters":{"class": ["Termo"]}
        }})
util_search.detail_document_found(doc_search_return)


Parâmetros usados: {'Retriever': {'top_k': 10, 'filters': {'class': ['Termo']}}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=2577, content='O termo é "Afastamento de responsável".
Afastamento de responsável tem definição: "No início ou no c...'>

Último (10):
<Document: id=83, content='O termo é "Anulação".
Anulação tem definição: "É o ato ou a decisão, de caráter judicial ou administ...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Afastamento de responsável', 0.9546168861748239]
1 : ['Risco de controle', 0.9431321899255821]
2 : ['Vazamento de dados', 0.942404112002669]
3 : ['Trancamento das contas', 0.9341491636096659]
4 : ['Revisão de ofício', 0.9297291992714615]
5 : ['Responsabilidade perante o controle ex

## Reranking - minilm_pt


mMiniLM-L6-v2-pt-msmarco-v2 is a multilingual miniLM-based model finetuned on a Portuguese translated version of MS MARCO passage dataset. In the v2 version, the Portuguese dataset was translated using Google Translate. Further information about the dataset or the translation method can be found on our mMARCO: A Multilingual Version of MS MARCO Passage Ranking Dataset and mMARCO repository.


In [67]:
nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_minilm = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
assert os.path.exists(nome_caminho_modelo_minilm), f"Path para {nome_caminho_modelo_minilm} não existe!"

In [56]:
#ranker_minilm_pt = SentenceTransformersRanker(model_name_or_path=nome_caminho_modelo)
ranker_minilm_pt = SentenceTransformersRankerLimit(model_name_or_path=nome_caminho_modelo_minilm, limit_query_size=350)

In [50]:
doctos_retornados_ranker = ranker_minilm_pt.predict(query=parm_query,documents = doc_search_return['documents'], top_k=5)
print(doctos_retornados_ranker[0],  doctos_retornados_ranker[0].meta['name'])
util_search.detail_document_found(doctos_retornados_ranker)

<Document: id=1110487, content='O termo é "Vazamento de dados".
Vazamento de dados tem definição: "Transmissão não-autorizada de dad...'> Vazamento de dados
1110487 0.0024509401991963387 Vazamento de dados
1000057 0.0007997120846994221 Termo de sigilo
2663 0.0002845456183422357 Trancamento das contas
1101311 0.00022979704954195768 Risco de controle
15291 0.00013715452223550528 Revisão de ofício


## Reranking - monot5_3b


In [59]:
nome_modelo_monot5_multilingual = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")
nome_caminho_modelo_monot5_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_multilingual
assert os.path.exists(nome_caminho_modelo_monot5_3b), f"Path para {nome_caminho_modelo_monot5_3b} não existe!"

In [53]:
ranker_monot5_multilingual = MonoT5RankerLimit(model_name_or_path=nome_caminho_modelo_monot5_3b, limit_query_size=350)

In [57]:
doctos_retornados_ranker = ranker_monot5_multilingual.predict(query=parm_query,documents = doc_search_return['documents'], top_k=5)
print(doctos_retornados_ranker[0],  doctos_retornados_ranker[0].meta['name'])
util_search.detail_document_found(doctos_retornados_ranker)

<Document: id=2663, content='O termo é "Trancamento das contas".
Trancamento das contas tem definição: "O Tribunal ordenará o tra...'> Trancamento das contas
2663 -6.416285991668701 Trancamento das contas
2577 -6.9923858642578125 Afastamento de responsável
1000057 -8.37442398071289 Termo de sigilo
5098 -8.556228637695312 Responsabilidade perante o controle externo
15291 -9.356241226196289 Revisão de ofício
1110487 -9.744372367858887 Vazamento de dados
6397 -9.853102684020996 Ato vinculado
1101311 -10.173725128173828 Risco de controle
15939 -10.493630409240723 Erro de procedimento
83 -11.372177124023438 Anulação


## Criar pipelines

In [61]:
import importlib
importlib.reload(util_elastic)

<module 'util.util_elastic' from '/home/borela/fontes/ind-ir/code/util/util_elastic.py'>

In [62]:
pipe_bm25_ranker_monot5_3b = util_elastic.return_pipeline_bm25_reranker(index, 'MONOT5', nome_caminho_modelo_monot5_3b)

In [64]:
doctos_retornados_ranker = pipe_bm25_ranker_monot5_3b.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=1101404, content='O termo é "Ofício de requisição".
Ofício de requisição tem definição: "Documento assinado por coorde...'>

Último (10):
<Document: id=5949, content='O termo é "Teletrabalho".
Teletrabalho tem definição: "Todo e qualquer trabalho realizado a distânci...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Ofício de requisição', -4.997889041900635]
1 : ['Julgamento de contas', -8.07536506652832]
2 : ['Evidência documental', -8.139838218688965]
3 : ['Processo administrativo de responsabilização', -9.24802017211914]
4 : ['Nexo de causalidade', -9.401777267456055]
5 : ['Competência do TCU', -9.931207656860352]
6 : ['Papéis de trabalho', -1

In [68]:
import importlib
importlib.reload(util_elastic)

<module 'util.util_elastic' from '/home/borela/fontes/ind-ir/code/util/util_elastic.py'>

In [69]:
pipe_bm25_ranker_minilm = util_elastic.return_pipeline_bm25_reranker(index, 'MINILM', nome_caminho_modelo_minilm)

In [71]:
doctos_retornados_ranker = pipe_bm25_ranker_minilm.run(query=parm_query)
util_search.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 10
Primeiro docto:
<Document: id=5095, content='O termo é "Competência do TCU". Competência do TCU tem definição: "São as seguintes as competências ...'>

Último (10):
<Document: id=4495, content='O termo é "Nexo de causalidade".
Nexo de causalidade tem definição: "Elemento que evidencia se a con...'>
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Competência do TCU', 0.0014646538766101003]
1 : ['Processo administrativo de responsabilização', 0.0004114470211789012]
2 : ['Evidência documental', 0.00033812460605986416]
3 : ['Plano de contratações', 0.0002457168884575367]
4 : ['Julgamento de contas', 0.00022130725847091526]
5 : ['Teletrabalho', 0.00016929447883740067]
6 : ['Ofício de